In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os

In [2]:
# Reading and merging all csv files into one dataframe

DATA_DIR = 'data/safety/features'

colnames = ['bookingID', 'Accuracy', 'Bearing', 'acceleration_x',
             'acceleration_y', 'acceleration_z', 'gyro_x', 'gyro_y',
             'gyro_z', 'second', 'Speed']
raw_df = pd.DataFrame(columns=colnames)

for filename in os.listdir(DATA_DIR):
    if filename !=  '.DS_Store':
        print(os.path.join(os.getcwd(), DATA_DIR, filename))
        new_df = pd.read_csv(os.path.join(os.getcwd(), DATA_DIR, filename))
        raw_df = pd.concat([raw_df, new_df], axis=0, ignore_index=True)
        print(raw_df.shape)

/Users/itn.muhammad.afif/Documents/notebooks/data/safety/features/part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
(1613558, 11)
/Users/itn.muhammad.afif/Documents/notebooks/data/safety/features/part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
(3227112, 11)
/Users/itn.muhammad.afif/Documents/notebooks/data/safety/features/part-00003-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
(4840665, 11)
/Users/itn.muhammad.afif/Documents/notebooks/data/safety/features/part-00002-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
(6454220, 11)
/Users/itn.muhammad.afif/Documents/notebooks/data/safety/features/part-00005-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
(8067771, 11)
/Users/itn.muhammad.afif/Documents/notebooks/data/safety/features/part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
(9681333, 11)
/Users/itn.muhammad.afif/Documents/notebooks/data/safety/features/part-00004-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
(11294892, 11)
/Users/itn.muhammad.afif/Documents/noteb

In [3]:
# Reading in labels csv

LABEL_DIR = os.path.join(os.getcwd(), 'data/safety/labels', 
                         'part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv')

label_df = pd.read_csv(LABEL_DIR)
label_df.head()

,bookingID,label
0,111669149733,0
1,335007449205,1
2,171798691856,0
3,1520418422900,0
4,798863917116,0


In [4]:
# Checking and dropping the 18 duplicate bookingIDs 

label_counts = np.unique(label_df['bookingID'].values, return_counts=True)
label_counts = pd.DataFrame({'bookingID' : label_counts[0], 
                             'counts' : label_counts[1]})

dup = label_counts[label_counts['counts'] > 1]

dup_id = dup['bookingID'].values

# bookingIDs are labelled both 0 and 1, drop these IDs since we have sufficient data

for b in dup_id:
    idx1 = label_df[label_df['bookingID'] == b].index
    idx2 = raw_df[raw_df['bookingID'] == b].index

    label_df = label_df.drop(idx1, axis=0)
    raw_df = raw_df.drop(idx2, axis=0)
    
df = pd.merge(raw_df, label_df)

In [5]:
# Dropping inaccurate readings 
# i.e. rows with accuracy > 30 and speed < 0

df = df.loc[(df['Accuracy'] <= 30) & (df['Speed'] >= 0)]

### Feature Engineering

Adding additional columns 
1. Distance = 'Speed' * 'second' 
2. Acceleration = sqrt('acceleration_x' ** 2, 'acceleration_y' ** 2, 'acceleration_z' ** 2) 
3. gyro_pc (magnitude) = PC1 of PCA applied on gyro_x, gyro_y, gyro_z
4. Speed_diff = per second difference in speed for each bookingID
5. Bearing_diff = per second difference in bearing for each bookingID

In [10]:
df_add = df.copy(deep=True).sort_values(by=['bookingID', 'second']).reset_index(drop=True)

# Applying PCA to gyro coordinates

from sklearn.decomposition import PCA

gyro_cols = ['gyro_x', 'gyro_y', 'gyro_z']
gyro_coord = df_add[gyro_cols]
pca = PCA()

pca.fit(gyro_coord)
pca.explained_variance_ratio_

array([0.75386447, 0.12894376, 0.11719177])

In [12]:
# Since first PC accounts for > 70% of variance, we only keep 1 PC to explain gyro

df_add['gyro_pc'] = pca.transform(df_add[gyro_cols])[:, 0]
df_add.head()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed,label,gyro_pc
0,0,12.0,143.298294,0.818112,-9.941461,-2.014999,-0.016245,-0.094040,0.070732,0.0,3.442991,0,-0.089042
1,0,8.0,143.298294,0.546405,-9.835590,-2.038925,-0.047092,-0.078874,0.043187,1.0,0.228454,0,-0.076595
2,0,8.0,143.298294,-1.706207,-9.270792,-1.209448,-0.028965,-0.032652,0.015390,2.0,0.228454,0,-0.032230
3,0,8.0,143.298294,-1.416705,-9.548032,-1.860977,-0.022413,0.005049,-0.025753,3.0,0.228454,0,0.002411
4,0,8.0,143.298294,-0.598145,-9.853534,-1.378574,-0.014297,-0.046206,0.021902,4.0,0.228454,0,-0.044964


In [13]:
# Adding distance, acceleration and magnitude columns

df_add['distance'] = df_add['Speed'] * df_add['second']
df_add['acceleration'] = np.sqrt(np.power(df_add['acceleration_x'], 2) + 
                                 np.power(df_add['acceleration_y'], 2) + 
                                 np.power(df_add['acceleration_z'], 2))

In [56]:
# Function to calculate difference in speed per second
def calc_speed_diff(x):
    return x['Speed'].diff()

# Function to calculate difference in bearing per second
def calc_bearing_diff(x):
    return x['Bearing'].diff()

# sub_df is a dataframe containing differences in speed and bearing
# values per second
# Large change in speed == sudden speeding/braking
# Large change in bearing == sudden change in direction/lane change

df_add['speed_diff'] = df_add.groupby('bookingID').apply(calc_speed_diff).fillna(method="backfill").values
df_add['bearing_diff'] = df_add.groupby('bookingID').apply(calc_bearing_diff).fillna(method="backfill").values

In [57]:
df_add.head(n=10)

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed,label,gyro_pc,distance,acceleration,speed_diff,bearing_diff
0,0,12.0,143.298294,0.818112,-9.941461,-2.014999,-0.016245,-0.094040,0.070732,0.0,3.442991,0,-0.089042,0.000000,10.176551,-3.214536,0.0
1,0,8.0,143.298294,0.546405,-9.835590,-2.038925,-0.047092,-0.078874,0.043187,1.0,0.228454,0,-0.076595,0.228454,10.059553,-3.214536,0.0
2,0,8.0,143.298294,-1.706207,-9.270792,-1.209448,-0.028965,-0.032652,0.015390,2.0,0.228454,0,-0.032230,0.456909,9.503762,0.000000,0.0
3,0,8.0,143.298294,-1.416705,-9.548032,-1.860977,-0.022413,0.005049,-0.025753,3.0,0.228454,0,0.002411,0.685363,9.830320,0.000000,0.0
4,0,8.0,143.298294,-0.598145,-9.853534,-1.378574,-0.014297,-0.046206,0.021902,4.0,0.228454,0,-0.044964,0.913818,9.967466,0.000000,0.0
5,0,8.0,143.298294,-0.608313,-9.539658,-1.794583,-0.007538,-0.023838,0.018068,5.0,0.228454,0,-0.022816,1.142272,9.726029,0.000000,0.0
6,0,8.0,143.298294,-0.867758,-9.698615,-1.615439,0.022728,-0.012178,0.005982,6.0,0.228454,0,-0.011493,1.370727,9.870449,0.000000,0.0
7,0,8.0,143.298294,-1.050790,-9.745270,-1.411771,0.027603,0.001841,0.000904,7.0,0.228454,0,0.002201,1.599181,9.902906,0.000000,0.0
8,0,8.0,143.298294,-0.721213,-9.960004,-1.202271,0.001864,-0.007702,0.014018,8.0,0.228454,0,-0.006844,1.827636,10.058194,0.000000,0.0
9,0,8.0,143.298294,-0.346924,-9.532629,-1.204663,0.014962,-0.050033,0.025118,9.0,0.228454,0,-0.047958,2.056090,9.614707,0.000000,0.0


In [106]:
def spread(x):
    return x.max() - x.min()

df_sub1 = df_add.groupby('bookingID')['acceleration', 'gyro_pc', 'Speed', 'second'].aggregate([
    'mean', 'median', 'std', spread
])

df_sub1.columns = ["_".join(x) for x in df_sub1.columns.ravel()]
df_sub1.columns = [col.lower() for col in df_sub1.columns]
df_sub1 = df_sub1.fillna(0)

In [107]:
df_sub1.head(n=10)

,acceleration_mean,acceleration_median,acceleration_std,acceleration_spread,gyro_pc_mean,gyro_pc_median,gyro_pc_std,gyro_pc_spread,speed_mean,speed_median,speed_std,speed_spread,second_mean,second_median,second_std,second_spread
bookingID,,,,,,,,,,,,,,,,
0,9.883337,9.852269,0.619492,6.530989,-0.006583,-0.002863,0.099002,1.101352,9.003204,8.503366,7.200000,22.881775,902.936128,1086.5,534.113894,1589.0
1,9.865608,9.847932,0.522142,5.819621,-0.006855,-0.003612,0.090770,1.123587,8.019369,7.206634,7.025981,21.882141,581.681384,607.5,289.129088,1034.0
2,9.929590,9.877755,0.515173,5.168422,-0.012751,0.001369,0.117109,0.896289,3.157213,2.998761,2.897762,9.360483,339.441026,97.0,356.319445,825.0
4,9.813434,9.791035,0.620066,13.349284,0.022429,0.024239,0.112628,1.166471,6.150996,3.310000,5.595901,19.780001,547.495430,547.5,315.962793,1094.0
6,9.918090,9.904142,0.585346,7.280114,0.000480,0.004189,0.106469,1.161481,4.628921,1.936962,5.314844,16.394695,547.000000,547.0,316.243577,1094.0
7,9.826470,9.789800,0.916836,8.572037,0.002651,-0.002687,0.072664,0.903745,12.176386,13.017325,8.680455,25.230654,480.947313,481.0,276.761488,959.0
8,9.763213,9.646309,0.730155,9.416841,-0.000840,0.000250,0.078446,0.754180,5.384260,3.540000,5.657824,18.270000,257.176623,268.0,130.510496,462.0
10,9.550677,9.494390,0.833292,9.474737,0.001922,-0.000612,0.110181,0.909695,8.702027,9.580000,7.002632,20.050000,187.000000,187.0,108.397417,374.0
11,9.948639,9.877962,0.750480,5.686104,-0.004018,-0.003111,0.151980,0.988519,6.659024,5.192059,6.019429,17.876741,132.000000,112.0,93.043769,299.0


In [141]:
print(df_add.columns)

# Checking 75th percentile for speed and second
# Checking 25th and 75th percentile for acceleration, gyro coordinates

outlier_vals = {'Speed_75' : np.quantile(df_add['Speed'], 0.75),
               'second_75' : np.quantile(df_add['second'], 0.75),
               'acceleration_x_25' : np.quantile(df_add['acceleration_x'], 0.25), 
               'acceleration_y_25' : np.quantile(df_add['acceleration_y'], 0.25),
               'acceleration_z_25' : np.quantile(df_add['acceleration_z'], 0.25),
               'acceleration_x_75' : np.quantile(df_add['acceleration_x'], 0.75),
               'acceleration_y_75' : np.quantile(df_add['acceleration_y'], 0.75),
               'acceleration_z_75' : np.quantile(df_add['acceleration_z'], 0.75),
               'gyro_x_25' : np.quantile(df_add['gyro_x'], 0.25),
               'gyro_y_25' : np.quantile(df_add['gyro_y'], 0.25),
               'gyro_z_25' : np.quantile(df_add['gyro_z'], 0.25),
               'gyro_x_75' : np.quantile(df_add['gyro_x'], 0.75),
               'gyro_y_75' : np.quantile(df_add['gyro_y'], 0.75),
               'gyro_z_75' : np.quantile(df_add['gyro_z'], 0.75)}

outlier_vals

Index(['bookingID', 'Accuracy', 'Bearing', 'acceleration_x', 'acceleration_y',
       'acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z', 'second', 'Speed',
       'label', 'gyro_pc', 'distance', 'acceleration', 'speed_diff',
       'bearing_diff'],
      dtype='object')


{'Speed_75': 15.67,
 'second_75': 862.0,
 'acceleration_x_25': -0.506329345703125,
 'acceleration_y_25': 0.2875244775,
 'acceleration_z_25': -0.8680572509765626,
 'acceleration_x_75': 0.6344828,
 'acceleration_y_75': 9.718361999999999,
 'acceleration_z_75': 2.7852156,
 'gyro_x_25': -0.026582342420625914,
 'gyro_y_25': -0.029722957,
 'gyro_z_25': -0.018615723,
 'gyro_x_75': 0.02321850364774683,
 'gyro_y_75': 0.031136672999999997,
 'gyro_z_75': 0.018109497}

In [146]:
def quantile_check(x, col, q):
    val = outlier_vals['{}_{}'.format(col, q)]
    
    if q == 25:
        return np.sum(x[col] < val)
    elif q == 75:
        return np.sum(x[col] > val)
    

df_sub2 = pd.DataFrame()
    
colnames = ['acceleration_x', 'acceleration_y', 'acceleration_z', 
           'gyro_x', 'gyro_y', 'gyro_z']

df_sub2['over_Speed'] = df_add.groupby('bookingID').apply(quantile_check, col='Speed', q=75)
df_sub2['over_second'] = df_add.groupby('bookingID').apply(quantile_check, col='second', q=75)

for col in colnames:
    print(col)
    arr = df_add.groupby('bookingID').apply(quantile_check, col=col, 
                                             q=25).values + df_add.groupby('bookingID').apply(quantile_check, 
                                                                                              col=col, q=75).values
    df_sub2['over_{}'.format(col)] = arr

acceleration_x
acceleration_y
acceleration_z
gyro_x
gyro_y
gyro_z


In [154]:
df_add2 = pd.merge(df_sub1, df_sub2, on='bookingID')
df_add2 = pd.merge(df_add2, label_df, on='bookingID')
df_add2.columns = [col.lower() for col in df_add2.columns]

In [100]:
# df_add2 = df_add.groupby('bookingID').aggregate({'second' : ['min', 'mean', 'max', 'median', 'std', spread], 
#                                                  'Speed' : ['min', 'mean', 'max', 'median', 'std', spread], 
#                                                 'distance' : ['min', 'mean', 'max', 'median', 'std', spread],
#                                                 'acceleration' : ['min', 'mean', 'max', 'median', 'std', spread],
#                                                 'gyro_pc' : ['min', 'mean', 'max', 'median', 'std', spread],
#                                                 'speed_diff' : ['min', 'mean', 'max', 'median', 'std', spread],
#                                                 'bearing_diff' : ['min', 'mean', 'max', 'median', 'std', spread]})

# df_add2.columns = ["_".join(x) for x in df_add2.columns.ravel()]
# df_add2 = pd.merge(df_add2, label_df, on='bookingID').drop('bookingID', axis='columns')
# df_add2.columns = [col.lower() for col in df_add2.columns]

In [155]:
df_add2.head(n=10)

,bookingid,acceleration_mean,acceleration_median,acceleration_std,acceleration_spread,gyro_pc_mean,gyro_pc_median,gyro_pc_std,gyro_pc_spread,speed_mean,...,second_spread,over_speed,over_second,over_acceleration_x,over_acceleration_y,over_acceleration_z,over_gyro_x,over_gyro_y,over_gyro_z,label
0,0,9.883337,9.852269,0.619492,6.530989,-0.006583,-0.002863,0.099002,1.101352,9.003204,...,1589.0,247,687,709,1002,801,570,581,614,0
1,1,9.865608,9.847932,0.522142,5.819621,-0.006855,-0.003612,0.090770,1.123587,8.019369,...,1034.0,157,168,483,270,790,230,404,347,1
2,2,9.929590,9.877755,0.515173,5.168422,-0.012751,0.001369,0.117109,0.896289,3.157213,...,825.0,0,0,102,143,34,67,103,71,1
3,4,9.813434,9.791035,0.620066,13.349284,0.022429,0.024239,0.112628,1.166471,6.150996,...,1094.0,122,232,389,1094,1072,531,735,753,1
4,6,9.918090,9.904142,0.585346,7.280114,0.000480,0.004189,0.106469,1.161481,4.628921,...,1094.0,18,232,463,407,257,533,450,561,0
5,7,9.826470,9.789800,0.916836,8.572037,0.002651,-0.002687,0.072664,0.903745,12.176386,...,959.0,365,97,809,262,219,561,490,687,0
6,8,9.763213,9.646309,0.730155,9.416841,-0.000840,0.000250,0.078446,0.754180,5.384260,...,462.0,25,0,357,94,36,157,159,82,0
7,10,9.550677,9.494390,0.833292,9.474737,0.001922,-0.000612,0.110181,0.909695,8.702027,...,374.0,90,0,300,71,307,90,111,149,0
8,11,9.948639,9.877962,0.750480,5.686104,-0.004018,-0.003111,0.151980,0.988519,6.659024,...,299.0,14,0,129,120,44,147,154,141,0
9,14,9.873517,9.823053,0.425662,5.916028,-0.002192,0.000388,0.082987,0.767631,4.152211,...,555.0,1,0,155,441,10,349,207,374,0


In [ ]:
# Checking for correlations within aggregated data
# Particularly, check for correlations between different features 
# e.g. strong correlation between accuracy and speed etc

corr_matrix = df_add2.corr()
corr_matrix.where((corr_matrix > 0.5) & (corr_matrix != 1))

In [ ]:
# def plot_corr_heatmap(df, vmax=1.0):
#     corr_matrix = df.corr()
    
#     mask = np.zeros_like(corr_matrix, dtype=np.bool)
#     mask[np.triu_indices_from(mask)] = True

#     # Set up the matplotlib figure
#     f, ax = plt.subplots(figsize=(11, 9))

#     # Generate a custom diverging colormap
#     cmap = sns.diverging_palette(220, 10, as_cmap=True)

#     # Draw the heatmap with the mask and correct aspect ratio
#     sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=vmax, 
#                 square=True, center=0, linewidths=.5)
    
# plot_corr_heatmap(df, vmax=0.6)

In [ ]:
# sns.pairplot(df_add2, hue='label')

In [156]:
SAVE_DIR = os.path.join(os.getcwd(), 'data/safety/')

df_add2.to_csv(SAVE_DIR + "kfengtee.csv", index=False)